In [1]:
import pandas as pd
import numpy as np
import random
import optuna

# -------------------------------PARTICLE SWARM OPTIMIZATION-------------------------------

## BASIC STEPS WHICH WERE FOLLOWED IN THE CODE:
### 1>CREATE RANDOM M PORTFOLIOS HAVING N ASSETS EACH
### 2>ASSIGN RADOM POSITIONS AND VELOCITES TO EACH OF THE ASSET IN EACH OF THE PORTFOLIO
### 3>ASSUME THAT THE FIRST PORTFOLIOTO BE THE  BEST AND LET IT BE GBEST.
### 4>THEN UPDATE THE POSTIONS OF ALL THE ASSETS IN A PORTFOLIO AND COMPARE WITH THEIR PREVIOUS POSTIONS  WRT SHARPE RATIO.
### 5>IF THE SHARPE RATIO OF THE UPDATED POSITION OF THE  PORTFOLIO IS GREATER THAN THE PREVIOUS POSITION OF THE PORTFOLIO,THEN   UPDATE THE   OLD POSITION WITH NEW POSITION AS IT IS HAVING HIGHER SHARPE RATIO,ELSE,NO CHANGES.
### 6>SAVE THE GREATER SHARE RATIO IN A LIST X.
### 7>REPEAT STEP 4,STEP 5,STEP 6 FOR ALL THE M PORTFOLIOS
### 8>AFTER ALL THE PORFOLIOS ARE COMPUTED,THEN FIND MAX(X) (X ,FROM STEP 5) AND THEN COMPARE MAX(X) WITH SHARPE RATIO OF THE GBEST PORTFOLIO(GBEST,FROM STEP 3)
### 9> IF THE SHARPE(MAX(X))>SHARPE(GBEST),UPDATE GBEST.
### 10>REPEAT STEP1 - STEP9 ,EXCEPT STEP3 BECAUSE WE NEED NOT HAVE TO ASSUME GBEST AS WE HAVE GOT GBEST FROM STEP 9,FOLLOW THE SAME PROCESS FOR P ITERATIONS, P IS USER CHOICE.
__

In [3]:
df = pd.read_csv("Nifty50.csv") 

In [4]:
df

,Date,Adj Close
0,2016-01-04,7791.299805
1,2016-01-05,7784.649902
2,2016-01-06,7741.000000
3,2016-01-07,7568.299805
4,2016-01-08,7601.350098
...,...,...
1476,2021-12-27,17086.250000
1477,2021-12-28,17233.250000
1478,2021-12-29,17213.599609
1479,2021-12-30,17203.949219


In [5]:
row=0
for date in range(len(df['Date'])):
    if df['Date'][date]!="2020-12-31":
        row+=1
    else:
        row+=1
        break        
df1=df.iloc[:row,:]
df1

,Date,Adj Close
0,2016-01-04,7791.299805
1,2016-01-05,7784.649902
2,2016-01-06,7741.000000
3,2016-01-07,7568.299805
4,2016-01-08,7601.350098
...,...,...
1228,2020-12-24,13749.250000
1229,2020-12-28,13873.200195
1230,2020-12-29,13932.599609
1231,2020-12-30,13981.950195


## INDEXING THE ASSETS

In [6]:
df1=list(df1.iloc[-1:])[1:]
stock_names={}
for i in range(len(df1)):
    stock_names[df1 [i]]=i+1
stock_names

{'Adj Close': 1}

In [8]:
n50=pd.read_csv('Nifty50.csv',parse_dates=['Date'],index_col='Date')
n50 = n50.iloc[:row,: ] 

trading_days=len(n50)
n50


,Adj Close
Date,
2016-01-04,7791.299805
2016-01-05,7784.649902
2016-01-06,7741.000000
2016-01-07,7568.299805
2016-01-08,7601.350098
...,...
2020-12-24,13749.250000
2020-12-28,13873.200195
2020-12-29,13932.599609


## CALCULATING THE NUMBER OF YEARS

In [9]:
start_date=n50.index[0]                 #date of first row in the dataset (datetime format)
end_date=n50.index[len(n50)-1]        #date of last row in the dataset  (datetime format)
no_of_years=int((((end_date-start_date).days+1)/365))#the difference give the number of total days (not trading days) over the total number of years in the dataset
no_of_years

4

In [10]:
close=[]
name=[]
stock_no=[]
daily_returns=[]
for i in n50:
    t=list(n50[i])#finding the close price col for each asset
    close.append(t)
    name.append(i)
    t=list(n50[i].pct_change())#daily return for stock
    daily_returns.append(t)


In [11]:
closed=[]
for i in close:
    for j in i:
        closed.append(j)
close=closed


In [12]:
daily_returns_arr=[]

for i in daily_returns:#HERE IN THIS CODE ,WE ARE TRYING TO PLACE ALL THE DAILY RETURNS IN A SINGLE LIST Daily_Returnss----><2>
    for j in i:
        daily_returns_arr.append(j)
daily_returns=daily_returns_arr

In [13]:
name_arr=[]

x=n50.shape[0] #FINDING NO OF ROWS, THE REASON IS EACH STOCK IS REPEATING X TIMES

for i in stock_names:                                                                    # ------><3>
    for j in range(x):
        name_arr.append(i)

name=name_arr

In [14]:
data={}
data['Close']=close
data['Daily_returns']=daily_returns
data['Name']=name
data=pd.DataFrame(data)
data

,Close,Daily_returns,Name
0,7791.299805,NaN,Adj Close
1,7784.649902,-0.000854,Adj Close
2,7741.000000,-0.005607,Adj Close
3,7568.299805,-0.022310,Adj Close
4,7601.350098,0.004367,Adj Close
...,...,...,...
1228,13749.250000,0.010893,Adj Close
1229,13873.200195,0.009015,Adj Close
1230,13932.599609,0.004282,Adj Close
1231,13981.950195,0.003542,Adj Close


In [15]:
data.isna().sum()

Close            5
Daily_returns    1
Name             0
dtype: int64

## REMOVING NAN VALUES

In [16]:
data=data.fillna(0)#.dropna(),it removes rows containing nan values

data

,Close,Daily_returns,Name
0,7791.299805,0.000000,Adj Close
1,7784.649902,-0.000854,Adj Close
2,7741.000000,-0.005607,Adj Close
3,7568.299805,-0.022310,Adj Close
4,7601.350098,0.004367,Adj Close
...,...,...,...
1228,13749.250000,0.010893,Adj Close
1229,13873.200195,0.009015,Adj Close
1230,13932.599609,0.004282,Adj Close
1231,13981.950195,0.003542,Adj Close


## DIMENSIONS REPRESENT THE NUMBER OF ASSETS

In [17]:
dimensions = len(stock_names)#No of stocks
#iterations = 500
#swarm_size = 100 #No of portfolios


# SHARPE

In [18]:

def SHARPE(weights, mean_daily_returns, cov_matrix):

    
    weights = [w/sum(weights) for w in weights] # Making sure all weights represent proportions that add up to 1
    weights = np.matrix(weights)
    port_return = np.round(np.sum(weights * mean_daily_returns.T) * trading_days, 2)/(no_of_years) # 1259 trading days over 5 year period
    port_std_dev = np.round(np.sqrt(weights * cov_matrix * weights.T) * np.sqrt(trading_days), 2)/np.sqrt(no_of_years)
    port_std_dev = float(port_std_dev)
    sharpe_ratio = (port_return - 0.0358)/port_std_dev   # 2.57 represents annual return of risk free security - 5-year US Treasury

    return sharpe_ratio

In [19]:
def PSO_HPTuning(swarm_size,iterations):
    mean_daily_returns = []  # IT WOULD CONTAIN THE AVG RETURS OF THE ASSETS
    all_return_input = [] #cov_input # IT WOULD CONTAIN THE [DAILY RETURNS] OF THE ASSETS
    neg_return=[]
    
    for stock in stock_names:  
        indv_stock = data[data.Name == stock]
        avg_return = indv_stock.Daily_returns.mean()
        mean_daily_returns.append(avg_return)
        all_return_input.append(indv_stock.Daily_returns.tolist())

    
    
    neg_return=all_return_input[:]
    mean_daily_returns = np.matrix(mean_daily_returns)  # CONVERTING 1D LIST(mean_daily_returns)INTO A MATRIX
    all_return_input = np.matrix(all_return_input)  # CONVERTING A 2D LIST(cov_input)INTO A MATRIX
    cov_matrix = np.cov(all_return_input)
    
    
    for i in range(len(neg_return)):
        for j in range(len(neg_return[i])):
            if neg_return[i][j]>0:
                neg_return[i][j]=0
    sortino_input= np.matrix(neg_return)
    cov_matrix_neg = np.cov(sortino_input)
    
    
    
    ''' w,c1,c2 ARE PARAMETERS WHICH WOULD HELP IN CHANGING THE PARTICLE(ASSET) POSITIONS'''
    '''INITIALIZING RANDOM POSITIONS(WEIGHTS) FOR ALL THE 100
    *11 ASSETS'''
    swarm_position = []
    swarm_velocity = []
    for particle in range(swarm_size):      #SWARM SIZE->PORTFOLIO             
        '''FOR EACH NEW PORTFOLIO, WE WOULD CREATING A LIST (position)'''
        position = [0]*dimensions   
        velocity = [0]*dimensions
        for dimension in range(dimensions):  #DIMENSIONS->ASSET
                                                    
            position[dimension] = random.random()  #random.random() would assign some random number between 0 and 1  
            velocity[dimension] = random.random()
        '''position LIST OF EVERY NEW PORTFOLIO WOULD BE KEPT IN A SEPARATE LIST(swarm_position)'''
        swarm_position.append(position)   
        swarm_velocity.append(velocity)
        
    swarm_position = np.array([np.array(p) for p in swarm_position]) # COVERTING  swarm_position  list to an array
    swarm_velocity = np.array([np.array(v) for v in swarm_velocity])  # COVERTING  swarm_velocities  list to an array
    
    '''initial_swarm_positions IS A LIST , CONTAINING ALL THE INITIAL POSITIONS OF ALL THE ASSETS OF ALL THE PORTFOLIOS'''
    initial_swarm_positions = swarm_position #HERE IS THE MASTER FILE WHICH WE WOULD BE USING FOR COMPARISON
    '''swarm_gbest,WE ARE ASSUMING THAT THE FIRST PORTFOLIO IS THE BEST OPTION AVAILABLE '''
    swarm_gbest = initial_swarm_positions [0] 
    
    avg_sharpe_list = []
    portfolio_return = []
    portfolio_standard_deviation = []
    portfolio_sortino = []
    portfolio_semi_deviation = []
    portfolio_sharpe = []
    portfolio_weights = []
    
    for iteration in range(iterations):
        '''FOR EACH ITERATION'''
        '''sharpe_pbest_all is a list which would contain the sharpe ratio of the portfolios,Finally at the end it would containing 100 sharpe ratios'''
        sharpe_pbest_all = []   
        for particle in range(swarm_size):#SWARM_SIZE->PORTFOLIOS 
            '''FOR EACH PORTFOLIO'''
            for dimension in range(dimensions):#DIMENSIONS->ASSETS
                '''FOR EACH ASSET'''
                '''HERE , WE WOULD BE UPDATING THE POSITIONS AND VELOCITES OF EACH OF THE ASSET PRESENT IN A PARTICULAR PORTFOLIO'''
                r1 = random.random() # random.random() WOULD BASICALLY GIVE YOU A VALUE BETWEEN 0 AND 1
                r2 = random.random()
                '''HERE WE WOULD BE CHANGING THE PRESENT PARTICLE(ASSET) POSITION 
                   AND FOR CHANGING THE POSITION,WE NEED TO CALCULATE THE NEW VELOCITY'''
    
                '''BASIC FORMULA 
                   NEW VELOCITY->w*PRESENT VELOCITY + c1 * r1 * abs(INTIAL POSITION - PRESENT POSITION)+c2*r2*abs(GBEST POSITION - PRESENT POSITION)
                   NEW POSITION->PRESENT POSITION + NEW VELOCITY
                '''
                w = (0.9 - 0.4)*((iterations - iteration)/iterations) + 0.4 # w at iteration, where initial_w = 0.9 & final_w = 0.4
                c1 = (0.5 - 2.5)*(iteration/iterations) + 2.5 # c1 at iteration, where min_c1 = 0.5 & max_c1 = 2.5
                c2 = (2.5 - 0.5)*(iteration/iterations) + 0.5 # c2 at iteration, where max_c2 = 2.5 & min_c2 = 0.5
                swarm_velocity[particle][dimension] = w * swarm_velocity[particle][dimension] + c1 * r1 * abs(initial_swarm_positions[particle][dimension] - swarm_position[particle][dimension]) + c2 * r2 * abs(swarm_gbest[dimension] - swarm_position[particle][dimension]) # Update velocity in every dimension
                swarm_position[particle][dimension] = swarm_position[particle][dimension] + swarm_velocity[particle][dimension] # Update position in every direction
            
            
            #AFTER 1 COMPLETE PORTFOLIO
            
            
            '''AFTER CHANGING THE POSITIONS OF ALL THE ASSETS IN A PARTICULAR PORTFOLIO, WE WOULD FIND THE SHARPE RATIO OF THIS PORTFOLIO
               AND WE WOULD STORE IT IN sharpe_pbest'''
            sharpe_pbest = SHARPE(initial_swarm_positions[particle], mean_daily_returns, cov_matrix) #Evaluating sharpe of existing pbest position
   
            '''THEN WE WOULD BE COMPARING THE sharpe_pbest WITH THE SHARPE RATIO OF THE INITIAL PORTFOLIO WHICH IS STORED IN initial_swarm_positions
               ie COMPARING SHARPE RATIO OF THE PORTFOLIO OLD POSITION WITH SHARPE RATIO OF THE PORTFOLIO NEW POSITION'''
        
            if SHARPE(swarm_position[particle], mean_daily_returns, cov_matrix) > sharpe_pbest:
                '''IS THE NEW POSITION BETTER THAN THE EXISTING ONE'''
                '''IF THE NEW POSITION IS BETTER, THEN UPDATE sharpe_pbest  and initial_swarm_positions[particle]'''
                initial_swarm_positions[particle] = swarm_position[particle] #Update pbest to new position
                sharpe_pbest = SHARPE(initial_swarm_positions[particle], mean_daily_returns, cov_matrix) #Update sharpe of pbest
            '''IF THE PORTFOLIO WITH NEW POSITION HAS HIGHER SHARPE RATIO THAN
               THE OLD POSITION,APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO NEW POSTION,
               ELSE APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO OLD POSTION '''
            sharpe_pbest_all.append(sharpe_pbest) 
            '''AFTER ALL THE PORTFOLIOS(ie,AFTER ONE ITERATION) ARE DONE, WE WOULD SELECT THE BEST PORTFOLIO
               WRT OF SHARPE RATIO AND COMPARE IT WITH THE SHARPE RATIO OF GLOBAL BEST(swarm_gbest PORTFOLIO)'''
            
            
        #AFTER 1 COMPLETE ITERATION
        
        
        if max(sharpe_pbest_all) > SHARPE(swarm_gbest, mean_daily_returns, cov_matrix): 
            ''' IS THE LARGEST SHARPE RATIO OF ALL THE PORTFOLIOS BETTER THEN gbest?'''
            ''' IF YES, CHANGE THE gbest to max(sharpe_pbest_all)'''
            max_index = sharpe_pbest_all.index(max(sharpe_pbest_all)) 
            swarm_gbest = initial_swarm_positions [max_index] 
    
    return SHARPE(swarm_gbest, mean_daily_returns, cov_matrix)
        
 

### SETTING THE HYPERPARAMETERS FROM EXCEL SHEET OF SURVEY OF PSO PAPERS

In [20]:
def objective(trial):
    swarm_size=trial.suggest_uniform('swarm_size',10,100)
    iterations=trial.suggest_uniform('iterations',100,500)
    return PSO_HPTuning(int(swarm_size),int(iterations))
    

In [21]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=50)

[I 2022-02-17 18:25:07,202] A new study created in memory with name: no-name-5bb56329-c580-46be-92ad-9344e9d1b114
[I 2022-02-17 18:25:23,119] Trial 0 finished with value: 0.6585 and parameters: {'swarm_size': 80.14434759672228, 'iterations': 422.0026345042828}. Best is trial 0 with value: 0.6585.
[I 2022-02-17 18:25:28,409] Trial 1 finished with value: 0.6585 and parameters: {'swarm_size': 22.699305898779926, 'iterations': 486.44844435350285}. Best is trial 0 with value: 0.6585.
[I 2022-02-17 18:25:36,820] Trial 2 finished with value: 0.6585 and parameters: {'swarm_size': 87.28395651200933, 'iterations': 192.50914358553155}. Best is trial 0 with value: 0.6585.
[I 2022-02-17 18:25:41,765] Trial 3 finished with value: 0.6585 and parameters: {'swarm_size': 40.7660794320247, 'iterations': 276.1985494196625}. Best is trial 0 with value: 0.6585.
[I 2022-02-17 18:25:51,927] Trial 4 finished with value: 0.6585 and parameters: {'swarm_size': 62.14123458936199, 'iterations': 356.7384422010066}. 

In [22]:
best=study.best_params

### BEST PARAMETERS FOR PSO

In [23]:
best

{'swarm_size': 80.14434759672228, 'iterations': 422.0026345042828}

In [24]:
swarm_size=int(best['swarm_size'])
iterations=int(best['iterations'])

## PSO FOR SHARPE RATIO

In [25]:
def PSO_OPTIMIZER(swarm_size,iterations):
    mean_daily_returns = []  # IT WOULD CONTAIN THE AVG RETURS OF THE ASSETS
    all_return_input = [] #cov_input # IT WOULD CONTAIN THE [DAILY RETURNS] OF THE ASSETS
    neg_return=[]
    
    for stock in stock_names:  
        indv_stock = data[data.Name == stock]
        avg_return = indv_stock.Daily_returns.mean()
        mean_daily_returns.append(avg_return)
        all_return_input.append(indv_stock.Daily_returns.tolist())

    
    
    neg_return=all_return_input[:]
    mean_daily_returns = np.matrix(mean_daily_returns)  # CONVERTING 1D LIST(mean_daily_returns)INTO A MATRIX
    all_return_input = np.matrix(all_return_input)  # CONVERTING A 2D LIST(cov_input)INTO A MATRIX
    cov_matrix = np.cov(all_return_input)
    
    
    for i in range(len(neg_return)):
        for j in range(len(neg_return[i])):
            if neg_return[i][j]>0:
                neg_return[i][j]=0
    sortino_input= np.matrix(neg_return)
    cov_matrix_neg = np.cov(sortino_input)
    
    
    
    ''' w,c1,c2 ARE PARAMETERS WHICH WOULD HELP IN CHANGING THE PARTICLE(ASSET) POSITIONS'''
    '''INITIALIZING RANDOM POSITIONS(WEIGHTS) FOR ALL THE 100
    *11 ASSETS'''
    swarm_position = []
    swarm_velocity = []
    for particle in range(swarm_size):      #SWARM SIZE->PORTFOLIO             
        '''FOR EACH NEW PORTFOLIO, WE WOULD CREATING A LIST (position)'''
        position = [0]*dimensions   
        velocity = [0]*dimensions
        for dimension in range(dimensions):  #DIMENSIONS->ASSET
                                                    
            position[dimension] = random.random()  #random.random() would assign some random number between 0 and 1  
            velocity[dimension] = random.random()
        '''position LIST OF EVERY NEW PORTFOLIO WOULD BE KEPT IN A SEPARATE LIST(swarm_position)'''
        swarm_position.append(position)   
        swarm_velocity.append(velocity)
        
    swarm_position = np.array([np.array(p) for p in swarm_position]) # COVERTING  swarm_position  list to an array
    swarm_velocity = np.array([np.array(v) for v in swarm_velocity])  # COVERTING  swarm_velocities  list to an array
    
    '''initial_swarm_positions IS A LIST , CONTAINING ALL THE INITIAL POSITIONS OF ALL THE ASSETS OF ALL THE PORTFOLIOS'''
    initial_swarm_positions = swarm_position #HERE IS THE MASTER FILE WHICH WE WOULD BE USING FOR COMPARISON
    '''swarm_gbest,WE ARE ASSUMING THAT THE FIRST PORTFOLIO IS THE BEST OPTION AVAILABLE '''
    swarm_gbest = initial_swarm_positions [0] 
    
    avg_sharpe_list = []
    portfolio_return = []
    portfolio_standard_deviation = []
    portfolio_sortino = []
    portfolio_semi_deviation = []
    portfolio_sharpe = []
    portfolio_weights = []
    
    for iteration in range(iterations):
        '''FOR EACH ITERATION'''
        '''sharpe_pbest_all is a list which would contain the sharpe ratio of the portfolios,Finally at the end it would containing 100 sharpe ratios'''
        sharpe_pbest_all = []   
        for particle in range(swarm_size):#SWARM_SIZE->PORTFOLIOS 
            '''FOR EACH PORTFOLIO'''
            for dimension in range(dimensions):#DIMENSIONS->ASSETS
                '''FOR EACH ASSET'''
                '''HERE , WE WOULD BE UPDATING THE POSITIONS AND VELOCITES OF EACH OF THE ASSET PRESENT IN A PARTICULAR PORTFOLIO'''
                r1 = random.random() # random.random() WOULD BASICALLY GIVE YOU A VALUE BETWEEN 0 AND 1
                r2 = random.random()
                '''HERE WE WOULD BE CHANGING THE PRESENT PARTICLE(ASSET) POSITION 
                   AND FOR CHANGING THE POSITION,WE NEED TO CALCULATE THE NEW VELOCITY'''
    
                '''BASIC FORMULA 
                   NEW VELOCITY->w*PRESENT VELOCITY + c1 * r1 * abs(INTIAL POSITION - PRESENT POSITION)+c2*r2*abs(GBEST POSITION - PRESENT POSITION)
                   NEW POSITION->PRESENT POSITION + NEW VELOCITY
                '''
                w = (0.9 - 0.4)*((iterations - iteration)/iterations) + 0.4 # w at iteration, where initial_w = 0.9 & final_w = 0.4
                c1 = (0.5 - 2.5)*(iteration/iterations) + 2.5 # c1 at iteration, where min_c1 = 0.5 & max_c1 = 2.5
                c2 = (2.5 - 0.5)*(iteration/iterations) + 0.5 # c2 at iteration, where max_c2 = 2.5 & min_c2 = 0.5
                swarm_velocity[particle][dimension] = w * swarm_velocity[particle][dimension] + c1 * r1 * abs(initial_swarm_positions[particle][dimension] - swarm_position[particle][dimension]) + c2 * r2 * abs(swarm_gbest[dimension] - swarm_position[particle][dimension]) # Update velocity in every dimension
                swarm_position[particle][dimension] = swarm_position[particle][dimension] + swarm_velocity[particle][dimension] # Update position in every direction
            
            
            #AFTER 1 COMPLETE PORTFOLIO
            
            
            '''AFTER CHANGING THE POSITIONS OF ALL THE ASSETS IN A PARTICULAR PORTFOLIO, WE WOULD FIND THE SHARPE RATIO OF THIS PORTFOLIO
               AND WE WOULD STORE IT IN sharpe_pbest'''
            sharpe_pbest = SHARPE(initial_swarm_positions[particle], mean_daily_returns, cov_matrix) #Evaluating sharpe of existing pbest position
   
            '''THEN WE WOULD BE COMPARING THE sharpe_pbest WITH THE SHARPE RATIO OF THE INITIAL PORTFOLIO WHICH IS STORED IN initial_swarm_positions
               ie COMPARING SHARPE RATIO OF THE PORTFOLIO OLD POSITION WITH SHARPE RATIO OF THE PORTFOLIO NEW POSITION'''
        
            if SHARPE(swarm_position[particle], mean_daily_returns, cov_matrix) > sharpe_pbest:
                '''IS THE NEW POSITION BETTER THAN THE EXISTING ONE'''
                '''IF THE NEW POSITION IS BETTER, THEN UPDATE sharpe_pbest  and initial_swarm_positions[particle]'''
                initial_swarm_positions[particle] = swarm_position[particle] #Update pbest to new position
                sharpe_pbest = SHARPE(initial_swarm_positions[particle], mean_daily_returns, cov_matrix) #Update sharpe of pbest
            '''IF THE PORTFOLIO WITH NEW POSITION HAS HIGHER SHARPE RATIO THAN
               THE OLD POSITION,APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO NEW POSTION,
               ELSE APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO OLD POSTION '''
            sharpe_pbest_all.append(sharpe_pbest) 
            '''AFTER ALL THE PORTFOLIOS(ie,AFTER ONE ITERATION) ARE DONE, WE WOULD SELECT THE BEST PORTFOLIO
               WRT OF SHARPE RATIO AND COMPARE IT WITH THE SHARPE RATIO OF GLOBAL BEST(swarm_gbest PORTFOLIO)'''
            
            
        #AFTER 1 COMPLETE ITERATION
        
        
        if max(sharpe_pbest_all) > SHARPE(swarm_gbest, mean_daily_returns, cov_matrix): 
            ''' IS THE LARGEST SHARPE RATIO OF ALL THE PORTFOLIOS BETTER THEN gbest?'''
            ''' IF YES, CHANGE THE gbest to max(sharpe_pbest_all)'''
            max_index = sharpe_pbest_all.index(max(sharpe_pbest_all)) 
            swarm_gbest = initial_swarm_positions [max_index] 
        
      
        '''AFTER 1 ITERATION IS DONE,ie GOING THROUGH ALL THE PORTFOLIOS AND FINDING THE BEST PORTFOLIO,
           WE WOULD NOW PRINT THE VALUES
        '''
        ''' CALCULATING THE AVG SHARPE RATIO FOR THE ITERATION '''
        avg_sharpe = sum(sharpe_pbest_all)/len(sharpe_pbest_all) 
        
        
        avg_sharpe_list.append(avg_sharpe)
        '''NOW FOR PRINTING THE IMPORTANT DATA FROM EACH ITERATION, WE HAVE CREATED A SEPARATE FUNCTION(PRINT_EACH_ITERATION)'''
        #portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino,  portfolio_weights = 
        #PRINT_EACH_ITERATION(iteration, swarm_gbest, mean_daily_returns, cov_matrix,cov_matrix_neg, portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino,  portfolio_weights)
        
        weights_arr = [w/sum(swarm_gbest) for w in swarm_gbest] # Making sure all weights represent proportions that add up to 1
        weights = np.matrix(weights_arr)
        port_return = np.round(np.sum(weights * mean_daily_returns.T) * trading_days, 2)/(no_of_years) # 1259 trading days over 5 year period
        port_std_dev = np.round(np.sqrt(weights * cov_matrix * weights.T) * np.sqrt(trading_days), 2)/np.sqrt(no_of_years)
        port_std_dev = float(port_std_dev)
        sharpe_ratio = (port_return - 0.0358)/port_std_dev   # 3.58 represents annual return of risk free security - 5-year US Treasury
        port_semi_dev = np.round(np.sqrt(weights * cov_matrix_neg * weights.T) * np.sqrt(trading_days), 2)/np.sqrt(no_of_years)
        port_semi_dev = float(port_semi_dev)
        sortino_ratio= (port_return-0.0358)/ (port_semi_dev)
        portfolio_return.append(port_return) # Adding portfolio return of a given PORTFOLIO  to  portfolio_return
        portfolio_standard_deviation.append(port_std_dev) # Adding portfolio standard deviation of a given PORTFOLIO to a list of standard deviations to portfolio_vol
        portfolio_sharpe.append(sharpe_ratio)# Adding portfolio sharpe ratio of a given PORTFOLIO  to   portfolio_sharpe
        portfolio_semi_deviation.append(port_semi_dev) # Adding portfolio standard deviation of a given PORTFOLIO to a list of standard deviations to portfolio_vol
        portfolio_sortino.append(sortino_ratio)# Adding portfolio sharpe ratio of a given PORTFOLIO  to   portfolio_sharpe
        portfolio_weights.append(weights_arr)# Adding portfolio weights of a given PORTFOLIO  to   portfolio_sharpe 

    

    #AFTER ALL THE ITERATIONS
    return portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino, portfolio_weights

In [26]:
avg_sharpe_list = []
portfolio_return = []
portfolio_standard_deviation = []
portfolio_sortino = []
portfolio_semi_deviation = []
portfolio_sharpe = []
portfolio_weights = []

In [27]:
portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation,portfolio_sortino, portfolio_weights = PSO_OPTIMIZER(swarm_size,iterations)

In [28]:
sharpe_portfolio = {'Returns' : portfolio_return, 'Standard Deviation' : portfolio_standard_deviation,  'Sharpe Ratio' : portfolio_sharpe}  

for counter,symbol in enumerate(n50.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in portfolio_weights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)

In [29]:
sharpe_pc.loc[:,:] *= 100
sharpe_pc.loc[:,'Sharpe Ratio'] /= 100

In [30]:
sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,Adj Close Weight
0,16.75,20.0,0.6585,100.0
1,16.75,20.0,0.6585,100.0
2,16.75,20.0,0.6585,100.0
3,16.75,20.0,0.6585,100.0
4,16.75,20.0,0.6585,100.0
...,...,...,...,...
417,16.75,20.0,0.6585,100.0
418,16.75,20.0,0.6585,100.0
419,16.75,20.0,0.6585,100.0
420,16.75,20.0,0.6585,100.0


In [31]:
sorted_sharpe=sharpe_pc.sort_values(by=['Sharpe Ratio'],ascending=False)


In [32]:
sorted_sharpe

,Returns,Standard Deviation,Sharpe Ratio,Adj Close Weight
0,16.75,20.0,0.6585,100.0
264,16.75,20.0,0.6585,100.0
288,16.75,20.0,0.6585,100.0
287,16.75,20.0,0.6585,100.0
286,16.75,20.0,0.6585,100.0
...,...,...,...,...
137,16.75,20.0,0.6585,100.0
136,16.75,20.0,0.6585,100.0
135,16.75,20.0,0.6585,100.0
134,16.75,20.0,0.6585,100.0


In [33]:
optimal_portfolio=sorted_sharpe.head(1)

In [34]:
optimal_portfolio.T

,0
Returns,16.7500
Standard Deviation,20.0000
Sharpe Ratio,0.6585
Adj Close Weight,100.0000


In [35]:
optimal_portfolio.T.to_csv('PSO(sharpe)_optimal_portfolio.csv')

In [36]:
top_10_portfolio=sorted_sharpe.head(10)

In [37]:
top_10_portfolio

,Returns,Standard Deviation,Sharpe Ratio,Adj Close Weight
0,16.75,20.0,0.6585,100.0
264,16.75,20.0,0.6585,100.0
288,16.75,20.0,0.6585,100.0
287,16.75,20.0,0.6585,100.0
286,16.75,20.0,0.6585,100.0
285,16.75,20.0,0.6585,100.0
284,16.75,20.0,0.6585,100.0
283,16.75,20.0,0.6585,100.0
282,16.75,20.0,0.6585,100.0
281,16.75,20.0,0.6585,100.0


In [38]:
top_10_portfolio.to_csv('PSO(sharpe)_top_10_portfolio.csv')

In [39]:
bottom_10_portfolio=sorted_sharpe.tail(10)

In [40]:
bottom_10_portfolio

,Returns,Standard Deviation,Sharpe Ratio,Adj Close Weight
142,16.75,20.0,0.6585,100.0
141,16.75,20.0,0.6585,100.0
140,16.75,20.0,0.6585,100.0
139,16.75,20.0,0.6585,100.0
138,16.75,20.0,0.6585,100.0
137,16.75,20.0,0.6585,100.0
136,16.75,20.0,0.6585,100.0
135,16.75,20.0,0.6585,100.0
134,16.75,20.0,0.6585,100.0
421,16.75,20.0,0.6585,100.0


In [41]:
bottom_10_portfolio.to_csv('PSO(sharpe)_bottom_10_portfolio.csv')

# SORTINO

In [42]:
def SORTINO(weights, mean_daily_returns,cov_matrix_neg):
    
    ''' MAKING SURE THAT ALL THE WEIGHTS OF THE PORTFOLIO EQUALS TO 1'''
    weights = [w/sum(weights) for w in weights]
    ''' CONVERTING weights TO A  MATRIX'''
    weights = np.matrix(weights)
    ''' CALCULATING THE PORTFOLIO RETURNS'''
    port_return = np.round(np.sum(weights * mean_daily_returns.T) *trading_days , 2)/no_of_years  #5yrs+91 days forecasted data from dhiraj's output,thus 5.25 yrs in total
                                                                                   

    port_semi_dev = np.round(np.sqrt(weights * cov_matrix_neg * weights.T) * np.sqrt(trading_days), 2)/np.sqrt(no_of_years)
    port_semi_dev = float(port_semi_dev)
    #referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
    #In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
    sortino_ratio= (port_return-0.0358)/ (port_semi_dev) #RFR FOR 2020->3.58%

    return sortino_ratio

In [43]:
def PSO_HPTuning_Sortino(swarm_size,iterations):
    mean_daily_returns = []  # IT WOULD CONTAIN THE AVG RETURS OF THE ASSETS
    all_return_input = [] #cov_input # IT WOULD CONTAIN THE [DAILY RETURNS] OF THE ASSETS
    neg_return=[]
    
    for stock in stock_names:  
        indv_stock = data[data.Name == stock]
        avg_return = indv_stock.Daily_returns.mean()
        mean_daily_returns.append(avg_return)
        all_return_input.append(indv_stock.Daily_returns.tolist())

    
    
    neg_return=all_return_input[:]
    mean_daily_returns = np.matrix(mean_daily_returns)  # CONVERTING 1D LIST(mean_daily_returns)INTO A MATRIX
    all_return_input = np.matrix(all_return_input)  # CONVERTING A 2D LIST(cov_input)INTO A MATRIX
    cov_matrix = np.cov(all_return_input)
    
    
    for i in range(len(neg_return)):
        for j in range(len(neg_return[i])):
            if neg_return[i][j]>0:
                neg_return[i][j]=0
    sortino_input= np.matrix(neg_return)
    cov_matrix_neg = np.cov(sortino_input)

    swarm_position = []
    swarm_velocity = []
    for particle in range(swarm_size):      #SWARM SIZE->PORTFOLIO             
        '''FOR EACH NEW PORTFOLIO, WE WOULD CREATING A LIST (position)'''
        position = [0]*dimensions   
        velocity = [0]*dimensions
        for dimension in range(dimensions):  #DIMENSIONS->ASSET
                                                    
            position[dimension] = random.random()  #random.random() would assign some random number between 0 and 1  
            velocity[dimension] = random.random()
        '''position LIST OF EVERY NEW PORTFOLIO WOULD BE KEPT IN A SEPARATE LIST(swarm_position)'''
        swarm_position.append(position)   
        swarm_velocity.append(velocity)
        
    swarm_position = np.array([np.array(p) for p in swarm_position]) # COVERTING  swarm_position  list to an array
    swarm_velocity = np.array([np.array(v) for v in swarm_velocity])  # COVERTING  swarm_velocities  list to an array
    
    initial_swarm_positions = swarm_position #HERE IS THE MASTER FILE WHICH WE WOULD BE USING FOR COMPARISON
    '''swarm_gbest,WE ARE ASSUMING THAT THE FIRST PORTFOLIO IS THE BEST OPTION AVAILABLE '''
    swarm_gbest = initial_swarm_positions [0] 
    
    avg_sortino_list = []
    portfolio_return = []
    portfolio_sortino = []
    portfolio_semi_deviation = []
    portfolio_weights = []
    
    for iteration in range(iterations):
        '''FOR EACH ITERATION'''
        '''sharpe_pbest_all is a list which would contain the sharpe ratio of the portfolios,Finally at the end it would containing 100 sharpe ratios'''
        sortino_pbest_all = []   
        for particle in range(swarm_size):#SWARM_SIZE->PORTFOLIOS 
            '''FOR EACH PORTFOLIO'''
            for dimension in range(dimensions):#DIMENSIONS->ASSETS
                '''FOR EACH ASSET'''
                '''HERE , WE WOULD BE UPDATING THE POSITIONS AND VELOCITES OF EACH OF THE ASSET PRESENT IN A PARTICULAR PORTFOLIO'''
                r1 = random.random() # random.random() WOULD BASICALLY GIVE YOU A VALUE BETWEEN 0 AND 1
                r2 = random.random()
                '''HERE WE WOULD BE CHANGING THE PRESENT PARTICLE(ASSET) POSITION 
                   AND FOR CHANGING THE POSITION,WE NEED TO CALCULATE THE NEW VELOCITY'''
    
                '''BASIC FORMULA 
                   NEW VELOCITY->w*PRESENT VELOCITY + c1 * r1 * abs(INTIAL POSITION - PRESENT POSITION)+c2*r2*abs(GBEST POSITION - PRESENT POSITION)
                   NEW POSITION->PRESENT POSITION + NEW VELOCITY
                '''
                w = (0.9 - 0.4)*((iterations - iteration)/iterations) + 0.4 # w at iteration, where initial_w = 0.9 & final_w = 0.4
                c1 = (0.5 - 2.5)*(iteration/iterations) + 2.5 # c1 at iteration, where min_c1 = 0.5 & max_c1 = 2.5
                c2 = (2.5 - 0.5)*(iteration/iterations) + 0.5 # c2 at iteration, where max_c2 = 2.5 & min_c2 = 0.5
                swarm_velocity[particle][dimension] = w * swarm_velocity[particle][dimension] + c1 * r1 * abs(initial_swarm_positions[particle][dimension] - swarm_position[particle][dimension]) + c2 * r2 * abs(swarm_gbest[dimension] - swarm_position[particle][dimension]) # Update velocity in every dimension
                swarm_position[particle][dimension] = swarm_position[particle][dimension] + swarm_velocity[particle][dimension] # Update position in every direction
            
            
            #AFTER 1 COMPLETE PORTFOLIO
            
            
            '''AFTER CHANGING THE POSITIONS OF ALL THE ASSETS IN A PARTICULAR PORTFOLIO, WE WOULD FIND THE SHARPE RATIO OF THIS PORTFOLIO
               AND WE WOULD STORE IT IN sharpe_pbest'''
            sortino_pbest = SORTINO(initial_swarm_positions[particle], mean_daily_returns, cov_matrix_neg) #Evaluating sharpe of existing pbest position
   
            '''THEN WE WOULD BE COMPARING THE sharpe_pbest WITH THE SHARPE RATIO OF THE INITIAL PORTFOLIO WHICH IS STORED IN initial_swarm_positions
               ie COMPARING SHARPE RATIO OF THE PORTFOLIO OLD POSITION WITH SHARPE RATIO OF THE PORTFOLIO NEW POSITION'''
        
            if SORTINO(swarm_position[particle], mean_daily_returns, cov_matrix_neg) > sortino_pbest:
                '''IS THE NEW POSITION BETTER THAN THE EXISTING ONE'''
                '''IF THE NEW POSITION IS BETTER, THEN UPDATE sharpe_pbest  and initial_swarm_positions[particle]'''
                initial_swarm_positions[particle] = swarm_position[particle] #Update pbest to new position
                sortino_pbest = SORTINO(initial_swarm_positions[particle], mean_daily_returns, cov_matrix_neg) #Update sharpe of pbest
            '''IF THE PORTFOLIO WITH NEW POSITION HAS HIGHER SHARPE RATIO THAN
               THE OLD POSITION,APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO NEW POSTION,
               ELSE APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO OLD POSTION '''
            sortino_pbest_all.append(sortino_pbest) 
            '''AFTER ALL THE PORTFOLIOS(ie,AFTER ONE ITERATION) ARE DONE, WE WOULD SELECT THE BEST PORTFOLIO
               WRT OF SHARPE RATIO AND COMPARE IT WITH THE SHARPE RATIO OF GLOBAL BEST(swarm_gbest PORTFOLIO)'''
            
            
        #AFTER 1 COMPLETE ITERATION
        
        
        if max(sortino_pbest_all) > SORTINO(swarm_gbest, mean_daily_returns, cov_matrix_neg): 

            max_index = sortino_pbest_all.index(max(sortino_pbest_all)) 
            swarm_gbest = initial_swarm_positions [max_index] 
    
    return SORTINO(swarm_gbest, mean_daily_returns, cov_matrix_neg)
        
 

In [44]:
def objective(trial):
    swarm_size=trial.suggest_uniform('swarm_size',10,100)
    iterations=trial.suggest_uniform('iterations',100,200)
    return PSO_HPTuning_Sortino(int(swarm_size),int(iterations))

## study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=50)

In [46]:
best=study.best_params

In [47]:
best

{'swarm_size': 57.00157138092797, 'iterations': 102.32249468251254}

In [48]:
swarm_size=int(best['swarm_size'])
iterations=int(best['iterations'])

In [49]:
def PSO_OPTIMIZER_SORTINO(swarm_size,iterations):
    
    mean_daily_returns = []  # IT WOULD CONTAIN THE AVG RETURS OF THE ASSETS
    all_return_input = [] #cov_input # IT WOULD CONTAIN THE [DAILY RETURNS] OF THE ASSETS
    neg_return=[]
    
    for stock in stock_names:  
        indv_stock = data[data.Name == stock]
        avg_return = indv_stock.Daily_returns.mean()
        mean_daily_returns.append(avg_return)
        all_return_input.append(indv_stock.Daily_returns.tolist())

    
    
    neg_return=all_return_input[:]
    mean_daily_returns = np.matrix(mean_daily_returns)  # CONVERTING 1D LIST(mean_daily_returns)INTO A MATRIX
    all_return_input = np.matrix(all_return_input)  # CONVERTING A 2D LIST(cov_input)INTO A MATRIX
    cov_matrix = np.cov(all_return_input)
    
    
    for i in range(len(neg_return)):
        for j in range(len(neg_return[i])):
            if neg_return[i][j]>0:
                neg_return[i][j]=0
    sortino_input= np.matrix(neg_return)
    cov_matrix_neg = np.cov(sortino_input)

    swarm_position = []
    swarm_velocity = []
    for particle in range(swarm_size):      #SWARM SIZE->PORTFOLIO             
        '''FOR EACH NEW PORTFOLIO, WE WOULD CREATING A LIST (position)'''
        position = [0]*dimensions   
        velocity = [0]*dimensions
        for dimension in range(dimensions):  #DIMENSIONS->ASSET
                                                    
            position[dimension] = random.random()  #random.random() would assign some random number between 0 and 1  
            velocity[dimension] = random.random()
        '''position LIST OF EVERY NEW PORTFOLIO WOULD BE KEPT IN A SEPARATE LIST(swarm_position)'''
        swarm_position.append(position)   
        swarm_velocity.append(velocity)
        
    swarm_position = np.array([np.array(p) for p in swarm_position]) # COVERTING  swarm_position  list to an array
    swarm_velocity = np.array([np.array(v) for v in swarm_velocity])  # COVERTING  swarm_velocities  list to an array
    
    initial_swarm_positions = swarm_position #HERE IS THE MASTER FILE WHICH WE WOULD BE USING FOR COMPARISON
    '''swarm_gbest,WE ARE ASSUMING THAT THE FIRST PORTFOLIO IS THE BEST OPTION AVAILABLE '''
    swarm_gbest = initial_swarm_positions [0] 
    
    avg_sortino_list = []
    portfolio_return = []
    portfolio_sortino = []
    portfolio_semi_deviation = []
    portfolio_weights = []
    
    for iteration in range(iterations):
        '''FOR EACH ITERATION'''
        '''sharpe_pbest_all is a list which would contain the sharpe ratio of the portfolios,Finally at the end it would containing 100 sharpe ratios'''
        sortino_pbest_all = []   
        for particle in range(swarm_size):#SWARM_SIZE->PORTFOLIOS 
            '''FOR EACH PORTFOLIO'''
            for dimension in range(dimensions):#DIMENSIONS->ASSETS
                '''FOR EACH ASSET'''
                '''HERE , WE WOULD BE UPDATING THE POSITIONS AND VELOCITES OF EACH OF THE ASSET PRESENT IN A PARTICULAR PORTFOLIO'''
                r1 = random.random() # random.random() WOULD BASICALLY GIVE YOU A VALUE BETWEEN 0 AND 1
                r2 = random.random()
                '''HERE WE WOULD BE CHANGING THE PRESENT PARTICLE(ASSET) POSITION 
                   AND FOR CHANGING THE POSITION,WE NEED TO CALCULATE THE NEW VELOCITY'''
    
                '''BASIC FORMULA 
                   NEW VELOCITY->w*PRESENT VELOCITY + c1 * r1 * abs(INTIAL POSITION - PRESENT POSITION)+c2*r2*abs(GBEST POSITION - PRESENT POSITION)
                   NEW POSITION->PRESENT POSITION + NEW VELOCITY
                '''
                w = (0.9 - 0.4)*((iterations - iteration)/iterations) + 0.4 # w at iteration, where initial_w = 0.9 & final_w = 0.4
                c1 = (0.5 - 2.5)*(iteration/iterations) + 2.5 # c1 at iteration, where min_c1 = 0.5 & max_c1 = 2.5
                c2 = (2.5 - 0.5)*(iteration/iterations) + 0.5 # c2 at iteration, where max_c2 = 2.5 & min_c2 = 0.5
                swarm_velocity[particle][dimension] = w * swarm_velocity[particle][dimension] + c1 * r1 * abs(initial_swarm_positions[particle][dimension] - swarm_position[particle][dimension]) + c2 * r2 * abs(swarm_gbest[dimension] - swarm_position[particle][dimension]) # Update velocity in every dimension
                swarm_position[particle][dimension] = swarm_position[particle][dimension] + swarm_velocity[particle][dimension] # Update position in every direction
            
            
            #AFTER 1 COMPLETE PORTFOLIO
            
            
            '''AFTER CHANGING THE POSITIONS OF ALL THE ASSETS IN A PARTICULAR PORTFOLIO, WE WOULD FIND THE SHARPE RATIO OF THIS PORTFOLIO
               AND WE WOULD STORE IT IN sharpe_pbest'''
            sortino_pbest = SORTINO(initial_swarm_positions[particle], mean_daily_returns, cov_matrix_neg) #Evaluating sharpe of existing pbest position
   
            '''THEN WE WOULD BE COMPARING THE sharpe_pbest WITH THE SHARPE RATIO OF THE INITIAL PORTFOLIO WHICH IS STORED IN initial_swarm_positions
               ie COMPARING SHARPE RATIO OF THE PORTFOLIO OLD POSITION WITH SHARPE RATIO OF THE PORTFOLIO NEW POSITION'''
        
            if SORTINO(swarm_position[particle], mean_daily_returns, cov_matrix_neg) > sortino_pbest:
                '''IS THE NEW POSITION BETTER THAN THE EXISTING ONE'''
                '''IF THE NEW POSITION IS BETTER, THEN UPDATE sharpe_pbest  and initial_swarm_positions[particle]'''
                initial_swarm_positions[particle] = swarm_position[particle] #Update pbest to new position
                sortino_pbest = SORTINO(initial_swarm_positions[particle], mean_daily_returns, cov_matrix_neg) #Update sharpe of pbest
            '''IF THE PORTFOLIO WITH NEW POSITION HAS HIGHER SHARPE RATIO THAN
               THE OLD POSITION,APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO NEW POSTION,
               ELSE APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO OLD POSTION '''
            sortino_pbest_all.append(sortino_pbest) 
            '''AFTER ALL THE PORTFOLIOS(ie,AFTER ONE ITERATION) ARE DONE, WE WOULD SELECT THE BEST PORTFOLIO
               WRT OF SHARPE RATIO AND COMPARE IT WITH THE SHARPE RATIO OF GLOBAL BEST(swarm_gbest PORTFOLIO)'''
            
            
        #AFTER 1 COMPLETE ITERATION
        
        
        if max(sortino_pbest_all) > SORTINO(swarm_gbest, mean_daily_returns, cov_matrix_neg): 

            max_index = sortino_pbest_all.index(max(sortino_pbest_all)) 
            swarm_gbest = initial_swarm_positions [max_index] 
            
        avg_sortino = sum(sortino_pbest_all)/len(sortino_pbest_all) 
        
        
        avg_sortino_list.append(avg_sortino)
        '''NOW FOR PRINTING THE IMPORTANT DATA FROM EACH ITERATION, WE HAVE CREATED A SEPARATE FUNCTION(PRINT_EACH_ITERATION)'''
        #portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino,  portfolio_weights = 
        #PRINT_EACH_ITERATION(iteration, swarm_gbest, mean_daily_returns, cov_matrix,cov_matrix_neg, portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino,  portfolio_weights)
        
        weights_arr = [w/sum(swarm_gbest) for w in swarm_gbest] # Making sure all weights represent proportions that add up to 1
        weights = np.matrix(weights_arr)
        port_return = np.round(np.sum(weights * mean_daily_returns.T) * trading_days, 2)/(no_of_years) # 1259 trading days over 5 year period
        port_std_dev = np.round(np.sqrt(weights * cov_matrix * weights.T) * np.sqrt(trading_days), 2)/np.sqrt(no_of_years)
        port_std_dev = float(port_std_dev)
        sharpe_ratio = (port_return - 0.0358)/port_std_dev   # 3.58 represents annual return of risk free security - 5-year US Treasury
        port_semi_dev = np.round(np.sqrt(weights * cov_matrix_neg * weights.T) * np.sqrt(trading_days), 2)/np.sqrt(no_of_years)
        port_semi_dev = float(port_semi_dev)
        sortino_ratio= (port_return-0.0358)/ (port_semi_dev)
        portfolio_return.append(port_return) # Adding portfolio return of a given PORTFOLIO  to  portfolio_return
        portfolio_standard_deviation.append(port_std_dev) # Adding portfolio standard deviation of a given PORTFOLIO to a list of standard deviations to portfolio_vol
        portfolio_sharpe.append(sharpe_ratio)# Adding portfolio sharpe ratio of a given PORTFOLIO  to   portfolio_sharpe
        portfolio_semi_deviation.append(port_semi_dev) # Adding portfolio standard deviation of a given PORTFOLIO to a list of standard deviations to portfolio_vol
        portfolio_sortino.append(sortino_ratio)# Adding portfolio sharpe ratio of a given PORTFOLIO  to   portfolio_sharpe
        portfolio_weights.append(weights_arr)# Adding portfolio weights of a given PORTFOLIO  to   portfolio_sharpe 

    

    #AFTER ALL THE ITERATIONS
    return portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino, portfolio_weights
        

In [50]:
avg_sharpe_list = []
portfolio_return = []
portfolio_standard_deviation = []
portfolio_sortino = []
portfolio_semi_deviation = []
portfolio_sharpe = []
portfolio_weights = []

In [51]:
portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation,portfolio_sortino, portfolio_weights = PSO_OPTIMIZER_SORTINO(swarm_size,iterations)

In [52]:
sortino_portfolio = {'Returns' : portfolio_return, 'Semi Deviation' : portfolio_semi_deviation,  'Sortino Ratio' : portfolio_sortino}  

for counter,symbol in enumerate(n50.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in portfolio_weights]
sortino_pc = pd.DataFrame(sortino_portfolio)

In [53]:
sortino_pc.loc[:,:] *= 100
sortino_pc.loc[:,'Sortino Ratio'] /= 100

In [54]:
sorted_sortino=sortino_pc.sort_values(by=['Sortino Ratio'],ascending=False)


In [55]:
sorted_sortino

,Returns,Semi Deviation,Sortino Ratio,Adj Close Weight
0,16.75,13.5,0.975556,100.0
64,16.75,13.5,0.975556,100.0
74,16.75,13.5,0.975556,100.0
73,16.75,13.5,0.975556,100.0
72,16.75,13.5,0.975556,100.0
...,...,...,...,...
31,16.75,13.5,0.975556,100.0
30,16.75,13.5,0.975556,100.0
29,16.75,13.5,0.975556,100.0
28,16.75,13.5,0.975556,100.0


In [56]:
optimal_portfolio_sortino=sorted_sharpe.head(1)

In [57]:
optimal_portfolio_sortino.T.to_csv('PSO(sortino)_optimal_portfolio.csv')

In [58]:
optimal_portfolio_sortino.T

,0
Returns,16.7500
Standard Deviation,20.0000
Sharpe Ratio,0.6585
Adj Close Weight,100.0000


In [59]:
top_10_portfolio_sortino=sorted_sortino.head(10)

In [60]:
top_10_portfolio_sortino

,Returns,Semi Deviation,Sortino Ratio,Adj Close Weight
0,16.75,13.5,0.975556,100.0
64,16.75,13.5,0.975556,100.0
74,16.75,13.5,0.975556,100.0
73,16.75,13.5,0.975556,100.0
72,16.75,13.5,0.975556,100.0
71,16.75,13.5,0.975556,100.0
70,16.75,13.5,0.975556,100.0
69,16.75,13.5,0.975556,100.0
68,16.75,13.5,0.975556,100.0
67,16.75,13.5,0.975556,100.0


In [61]:
top_10_portfolio_sortino.to_csv('PSO(sortino)_top_10_portfolio.csv')

In [62]:
top_10_portfolio_sortino

,Returns,Semi Deviation,Sortino Ratio,Adj Close Weight
0,16.75,13.5,0.975556,100.0
64,16.75,13.5,0.975556,100.0
74,16.75,13.5,0.975556,100.0
73,16.75,13.5,0.975556,100.0
72,16.75,13.5,0.975556,100.0
71,16.75,13.5,0.975556,100.0
70,16.75,13.5,0.975556,100.0
69,16.75,13.5,0.975556,100.0
68,16.75,13.5,0.975556,100.0
67,16.75,13.5,0.975556,100.0


In [63]:
bottom_10_portfolio_sortino=sorted_sortino.tail(10)

In [64]:
bottom_10_portfolio_sortino.to_csv('PSO(sortino)_bottom_10_portfolio.csv')

In [65]:
bottom_10_portfolio_sortino

,Returns,Semi Deviation,Sortino Ratio,Adj Close Weight
36,16.75,13.5,0.975556,100.0
35,16.75,13.5,0.975556,100.0
34,16.75,13.5,0.975556,100.0
33,16.75,13.5,0.975556,100.0
32,16.75,13.5,0.975556,100.0
31,16.75,13.5,0.975556,100.0
30,16.75,13.5,0.975556,100.0
29,16.75,13.5,0.975556,100.0
28,16.75,13.5,0.975556,100.0
101,16.75,13.5,0.975556,100.0
